#MULTIMODEL GENRATIVE MODEL

In [1]:
!pip install langchain

In [3]:
!pip install langchain-google-genai

In [4]:
!pip install gtts playsound SpeechRecognition

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.9 MB/s eta 0:00:00
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=4a8912c1a88909875fc50119c98de88caaad98b0243eb2ae3cf8153be3396995
  Stored in directory: /root/.cache/pip/wheels/cf/42/ff/7c587bae55eec67b909ca316b250d9b4daedbf272a3cbeb907
Successfully built playsound
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [7]:
!pip install google-generativeai

In [8]:
!pip install pydub

In [9]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 9.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=8bc2e7cdb67158f79a49088c689afe7982abc696d3b2fd638615c96f76447c2a
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


# AUDIO CHATBOT

In [64]:
import os
import numpy as np
from getpass import getpass
from google.colab import files
import warnings
warnings.filterwarnings("ignore")

In [65]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser

In [66]:
from pydub import AudioSegment
from gtts import gTTS
import whisper

In [67]:
os.environ["GEMINI_API"]= getpass("enter your api_key")

enter your api_key··········


In [91]:
upload = files.upload()
file_name = list(upload.keys())[0]

Saving tamil_audio_query.opus to tamil_audio_query.opus


audio-text conversion

In [92]:
model = whisper.load_model("base")
text = model.transcribe(file_name)
question = text["text"]
print(question)


 முது விணில் மாதத்தில் தகாலி முழ்கசடியில் அப்பரம் ஜோலா, இதில்லா வலப்பாங்கள்


LANGchain model

In [93]:
audio_prompt="""
You are a friendly voice-based farming assistant for Indian farmers.

Your role:
- Listen to the farmer’s question (already converted to text).
- Provide clear, practical, and farmer-friendly advice.
- Focus on farming topics: crop selection, sowing time, soil health, irrigation, pest control, fertilizers, organic methods, and weather.
- Keep answers short, simple, and easy to understand.
- Respond in the **same language** the farmer used in their question.
- answer within 150 words
-conversation so  far {history}

Farmer’s Question: {input}

 Answer:
"""

In [94]:
audio_llm = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", google_api_key=os.environ["GEMINI_API"])

In [95]:
audio_template = PromptTemplate(
    input_variables = ["history","input"],
    template = audio_prompt
)

In [96]:
audio_memory = ConversationBufferMemory(k=30)


In [97]:
audio_chain = ConversationChain(
    llm =audio_llm,
    prompt = audio_template,
    memory = audio_memory,
    output_parser=StrOutputParser())

In [98]:
answer = audio_chain.run({
    "input":question
})
print(answer)

வணக்கம்! உங்கள் தக்காளி செடிகளில் வாடல் அல்லது அழுகல் பிரச்சனை இருப்பதாக தெரிகிறது. இதற்கு பல காரணங்கள் இருக்கலாம்.

முதலில், செடிகளுக்கு அதிக தண்ணீர் பாய்ச்சுவதை தவிர்க்கவும். மண்ணில் வடிகால் வசதி சரியாக உள்ளதா என்று பார்க்கவும். தண்ணீர் தேங்காமல் பார்த்துக் கொள்ளுங்கள்.

செடிகள் இளம் நாற்றுகளாக இருந்தால், இது 'கருகல் நோய்' ஆக இருக்கலாம். இதற்கு விதைகளை விதைக்கும் முன் பூஞ்சாணக் கொல்லி கொண்டு விதை நேர்த்தி செய்யலாம்.

பெரிய செடிகளில் வாடல் இருந்தால், அது வேர் அழுகல் அல்லது வேறு பூஞ்சை நோயாக இருக்கலாம். பாதிக்கப்பட்ட செடிகளை அகற்றி, அருகில் உள்ள செடிகளுக்கு பூஞ்சாணக் கொல்லி தெளிக்கலாம் அல்லது பஞ்சகவ்யா போன்ற இயற்கை உரங்களை பயன்படுத்தலாம்.

சரியான காரணத்தை கண்டறிய, உங்கள் பகுதி வேளாண்மை அதிகாரியை அணுகுவது நல்லது.


#text to audio conversion

In [76]:
!pip install langdetect

In [99]:
from langdetect import detect

In [100]:
lang = detect(answer)
tts = gTTS(answer, lang = lang)
tts.save("farming_answer.mp3")

In [101]:
from IPython.display import Audio, display

In [102]:
display(Audio("farming_answer.mp3", autoplay=True))